# Analysis of RNASeq Data from Day 0 
In this notebook, I will be completing RNASeq analysis for the day 0 data. These efforts are part of Aim 1 for my BMEG490B paper, which is: 

<b> Characterizing T-Lineage Restriction of MLPs grown in LEM </b>

In this document, I will be reading in the created ReadCountdata, performing quality control, linking genes to one of three phenotypes of interest (CD45RA-CLEC12A-,CD45RA+CLEC12A- and CLEC12A+). 

## Experimental Workflow 

The experimental workflow generating this data comes from the files testting testing 